In [ ]:
import json
import requests

# Ryan code starts

In [ ]:
# url = "https://api.covid19api.com/all"
# response = requests.get(url)
# with open("covid_all.json", 'w') as jsonfile:
#     json.dump(response.json(), jsonfile)

In [ ]:
with open("static/data/covid_all.json") as f:
    covid = json.load(f)

In [ ]:
code to store data in MongoDB
import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.covid_db
for i in range(len(covid)):
    db.historical.insert_one(covid[i])

In [ ]:
confirmed = []
deaths = []
recovered = []

In [ ]:
for i in range(len(covid)):
    data = covid[i]
    if data['Status'] == 'confirmed':
        confirmed.append(data)
    elif data['Status'] == 'deaths':
        deaths.append(data)
    elif data['Status'] == 'recovered':
        recovered.append(data)

In [ ]:
def get_date_list(list):
    dates = []
    for i in range(len(list)):
        if list[i]['Date'] not in dates:
            dates.append(list[i]['Date'])
    return sorted(dates)

In [ ]:
dates_confirmed = []
dates_deaths = []
dates_recovered = []
dates_confirmed = get_date_list(confirmed)
dates_deaths = get_date_list(deaths)
dates_recovered = get_date_list(recovered)

In [ ]:
covid_clean = {
    'confirmed': {},
    'deaths': {},
    'recovered': {}    
}

In [ ]:
def get_date_data(dates, data_list, status):
    for i in range(len(dates)):
        current = []
        for j in range(len(data_list)):
            if dates[i] == data_list[j]["Date"]:
                current.append(data_list[j])
        covid_clean[status][dates[i]] = current
    return covid_clean

In [ ]:
covid_clean = get_date_data(dates_confirmed, confirmed, 'confirmed')
covid_clean = get_date_data(dates_deaths, deaths, 'deaths')
covid_clean = get_date_data(dates_recovered, recovered, 'recovered')

In [ ]:
# covid_clean
# for i in range(len(dates_deaths)):
#     print(len(covid_clean['deaths'][dates_deaths[i]]))

In [ ]:
# import pymongo
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)
# db = client.covid_db
# db.covid_clean.insert_one(covid_clean)  ## this is not working due to oversize

# db.covid_confirmed.insert_one(covid_clean['confirmed'])
# db.covid_deaths.insert_one(covid_clean['deaths'])
# db.covid_recovered.insert_one(covid_clean['recovered'])

# Ryan code ends

# Kana code starts

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
# Read unemployment claims data
claim_data = pd.read_csv("resources/unemployment_by_state.csv")
claim_data.head()

In [ ]:
# Select needed columns
claim_data = claim_data[['State','Filed week ended','Initial Claims']]
claim_data.head()

In [ ]:
# Replace comma with empty space
claim_data = claim_data.stack().str.replace(',','').unstack()
claim_data.head()

In [ ]:
# Retrive data only after 1/1/2020 - original date format doesn't allow me to sort!
filtered_claim_data = claim_data.loc[claim_data["Filed week ended"] > "1/1/2020"]
filtered_claim_data.head()

In [ ]:
# Change the date format to be able to sort
claim_data['Filed week ended'] = pd.to_datetime(claim_data['Filed week ended'])
# claim_data['Filed week ended date'] = claim_data['Filed week ended'].dt.date
claim_data.head()

# df['date_only'] = df['date_time_column'].dt.date

In [ ]:
# Retrive data only after 1/1/2020
filtered_claim_data = claim_data.loc[claim_data["Filed week ended"] > "2020-01-01"]

# Drop na
filtered_claim_data = filtered_claim_data.dropna()
filtered_claim_data.head()

In [ ]:
# Change data type to int
filtered_claim_data['Initial Claims'] = filtered_claim_data['Initial Claims'].astype(int)

# Check data type again
filtered_claim_data.dtypes

In [ ]:
# states = list(set(filtered_claim_data["State"]))
# states= sorted(states)

dates = list(set(filtered_claim_data["Filed week ended"]))
dates = sorted(dates)
dates

In [ ]:
# states_clean = {}
# for i in states:
#     states_clean.update({i:[]})
# states_clean
dates_clean = {}
for i in dates:
    dates_clean.update({i:[]})
dates_clean

In [ ]:
# for i in states:
#     current = []
#     for j in range(len(filtered_claim_data['State'])):
#         if filtered_claim_data['State'].iloc[j] == i:
#             state_claim_dict = {}
#             state_claim_dict[filtered_claim_data['Filed week ended'][j]] =filtered_claim_data['Initial Claims'][j]
#             current.append(state_claim_dict)

In [ ]:
# for key, value in states_clean.items():
#     current = []
#     for j in range(len(filtered_claim_data['State'])):
#         if filtered_claim_data['State'].iloc[j] == key:
#             state_claim_dict = {}
#             state_claim_dict[filtered_claim_data['Filed week ended'][j]] =filtered_claim_data['Initial Claims'][j]
#             current.append(state_claim_dict)
#             states_clean[key] = current
# print(states_clean)

for key, value in dates_clean.items():
    current = []
    for j in range(len(filtered_claim_data['Filed week ended'])):
        if filtered_claim_data['Filed week ended'].iloc[j] == key:
            state_claim_dict = {}
            print(key)
#             print(filtered_claim_data['Filed week ended'].iloc[j] )
        
#             print(state_claim_dict[filtered_claim_data['State'].iloc[j]])
            print(filtered_claim_data['Initial Claims'][j])
            state_claim_dict[filtered_claim_data['State'].iloc[j]] =filtered_claim_data['Initial Claims'][j]
#             current.append(state_claim_dict)
#             dates_clean[key] = current
# print(dates_clean)

# Kana code ends

# Connor code starts

# Connor code ends

# Nabeel code starts

# Nabeel code ends